In [ ]:
import gzip
import numpy as np
import os
import pickle

In [ ]:
def print_scores(path_result, sep=' ' * 4):
    key_list = ['ll', 'oca']
    with gzip.open(path_result, 'rb') as f:
        metrics_all = pickle.load(f)
        scores_mean, scores_std = {}, {}
        for key, val in metrics_all.items():
            if key not in key_list:
                continue
            if isinstance(val, tuple):
                assert len(val) == 2
                val = val[0].mean(-1) / val[1].mean(-1)
            else:
                val = val.mean(-1)
            scores_mean[key], scores_std[key] = val.mean(), val.std()
    print('LL:{:7.1f} {:.1e}'.format(scores_mean['ll'], scores_std['ll']), end=sep)
    print('OCA:{:6.3f} {:.1e}'.format(scores_mean['oca'], scores_std['oca']))
    return

folder_base = 'outs'

In [ ]:
# Extrapolation
folder = 'mnist_extrapol'
print(folder)
print('0, 1, 2 -> 0, 1, 2')
print_scores(os.path.join(folder_base, folder, 'test.pkl'))
print('0, 1, 2 -> 3')
print_scores(os.path.join(folder_base, folder, 'general.pkl'))

In [ ]:
# Interpolation
folder = 'mnist_interpol'
print(folder)
print('0, 1, 3 -> 0, 1, 3')
print_scores(os.path.join(folder_base, folder, 'test.pkl'))
print('0, 1, 3 -> 2')
print_scores(os.path.join(folder_base, folder, 'general.pkl'))